# Chargement des données et séparation des features et target

Avant tout préprocessing, on établit la séparation des données de train et de test afin de limiter la possibilité de data leak entre les jeux de données de train et de test.

In [1]:
import pandas as pd
import numpy as np
from collections import Counter

from sklearn.model_selection import train_test_split
from imblearn.under_sampling import RandomUnderSampler

X = pd.read_csv("../data/X_train_update.csv", index_col = 0)
y = pd.read_csv("../data/Y_train_CVw08PX.csv", index_col = 0)

# Split train/test
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

On sait que la classe 2583 (accessoires de piscines) est sur représentée dans le jeu de données. Nous avons décidé de faire de l'undersampling aléatoire pour diminuer sa présence à 6% au lieu de 12% afin d'équilibrer les classes présentes et améliorer les performances de l'apprentissage.

In [2]:
print("Avant undersampling (train) :")
print(y_train['prdtypecode'].value_counts(normalize=True) * 100)

# Calcul des effectifs
counts = Counter(y_train['prdtypecode'])
n_total = len(y_train)

# Cible = 6% pour 2583
target_ratio = 0.06
n_target_2583 = int(n_total * target_ratio)

# Construction d'une sampling_strategy
sampling_strategy = {cls: count for cls, count in counts.items()}
sampling_strategy[2583] = n_target_2583

# Application de l'undersampling
rus = RandomUnderSampler(sampling_strategy=sampling_strategy, random_state=42)
X_train, y_train = rus.fit_resample(X_train, y_train)

print("\nAprès undersampling (train) :")
print(y_train['prdtypecode'].value_counts(normalize=True) * 100)

print("\nTest set reste inchangé :")
print(y_test['prdtypecode'].value_counts(normalize=True) * 100)

Avant undersampling (train) :
2583    12.022316
1560     5.973621
1300     5.941235
2060     5.879409
2522     5.874993
1280     5.735147
2403     5.621798
2280     5.605606
1920     5.066832
1160     4.654655
1320     3.817052
10       3.669846
2705     3.251781
1140     3.145793
2582     3.048637
40       2.952953
2585     2.939704
1302     2.933816
1281     2.437732
50       1.979921
2462     1.673733
2905     1.027498
60       0.980392
2220     0.970088
1301     0.950951
1940     0.945063
1180     0.899429
Name: prdtypecode, dtype: float64

Après undersampling (train) :
2583    6.383145
1560    6.356516
1300    6.322055
2060    6.256266
2522    6.251566
1280    6.102757
2403    5.982143
2280    5.964912
1920    5.391604
1160    4.953008
1320    4.061717
10      3.905075
2705    3.460213
1140    3.347431
2582    3.244048
40      3.142231
2585    3.128133
1302    3.121867
1281    2.593985
50      2.106830
2462    1.781015
2905    1.093358
60      1.043233
2220    1.032268
1301    1.0

/home/cramarokoto/miniconda3/envs/py39/lib/python3.9/site-packages/sklearn/base.py:484: FutureWarning: `BaseEstimator._check_n_features` is deprecated in 1.6 and will be removed in 1.7. Use `sklearn.utils.validation._check_n_features` instead.
  warnings.warn(
/home/cramarokoto/miniconda3/envs/py39/lib/python3.9/site-packages/sklearn/base.py:493: FutureWarning: `BaseEstimator._check_feature_names` is deprecated in 1.6 and will be removed in 1.7. Use `sklearn.utils.validation._check_feature_names` instead.
  warnings.warn(


# Preprocessing des données

In [3]:
from scipy.sparse import hstack

# Traitement des variables textuelles
from sklearn.feature_extraction.text import TfidfVectorizer
import nltk
nltk.download('punkt_tab')
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem.snowball import FrenchStemmer

# Stopwords

html_stopwords = [
    'html', 'head', 'body', 'div', 'span', 'p', 'br', 'a', 'img', 'ul', 'li', 'ol', 'table',
    'tr', 'td', 'th', 'h1', 'h2', 'h3', 'h4', 'h5', 'h6', 'b', 'i', 'u', 'strong', 'em',
    'eacute', 'agrave'
]
punctuation_words = [",", ".", "``", "@", "*", "(", ")", "...", "!", "?", "-", 
                  "_", ">", "<", ":", "/", "=", "--", "©", "~", ";", "\\", "\\\\"]
final_stopwords = stopwords.words('english') + stopwords.words('french') + html_stopwords + punctuation_words


# Stemming and processing

stemmer = FrenchStemmer()

def stemming(mots) :
    sortie = []
    for string in mots :
        radical = stemmer.stem(string)
        if (radical not in sortie) : sortie.append(radical)
    return sortie


def preprocessing(text, with_stemming=False):
    text = text.lower()
    tokens = word_tokenize(text)
    result = [word for word in tokens if word not in final_stopwords]
    if with_stemming:
        result = stemming(result)
    return ' '.join(result)


[nltk_data] Downloading package punkt_tab to
[nltk_data]     /home/cramarokoto/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


## Preprocessing des données textuelles

### Création de la variable `has_description` basée sur la présence de la variable `description`

Nous avons vu lors de l'exploration que la répartition de la présence de description n'est pas homogène parmi les catégories de produits. On peut en déduire que l'absence ou présence de description est une information à part entière qu'il peut être intéressant d'exploiter dans nos modèles.

Nous décidons de créer la variable `has_description` qui vaut :
- 1 si la description est présente
- 0 sinon

In [4]:
# Pour X_train
X_train['has_description'] = X_train['description'].notnull() & (X_train['description'].str.strip() != "")
X_train['has_description'] = X_train['has_description'].astype(int)

# Pour X_test
X_test['has_description'] = X_test['description'].notnull() & (X_test['description'].str.strip() != "")
X_test['has_description'] = X_test['has_description'].astype(int)

### Fusion des variables `designation` et `description` dans `full_description`

La `description` et la `designation` sont toutes les deux des valeurs textuelles libres. Bien que `designation` soit plus courte, elles apportent le même type d'information sur le produit c'est-à-dire sa description et son appellation commune. `description` pouvant être nulle, cela peut être gênant pour son intégration dans les modèles. 

Afin de faciliter le traitement de ces deux valeurs textuelles, nous décidons de les fusionner dans une variable nommée `full_description`

In [5]:
# Pour X_train
X_train['full_description'] = (
    X_train['designation'] + " " + X_train['description'].fillna('')
).str.strip()
# Nettoyage des colonnes inutiles
X_train = X_train.drop(columns=['designation', 'description'])

# Pour X_test
X_test['full_description'] = (
    X_test['designation'] + " " + X_test['description'].fillna('')
).str.strip()
# Nettoyage des colonnes inutiles
X_test = X_test.drop(columns=['designation', 'description'])

### Nettoyage, tokenisation et stemmatisation de la variable `full_description`

In [6]:
result = pd.DataFrame(columns=['preprocessed_full_description', 'full_description'])

for i in X_train.head(10).index:
    current_description = X_train.loc[i, 'full_description']
    result.loc[i, 'full_description'] = current_description
    result.loc[i, 'preprocessed_full_description'] = preprocessing(current_description, with_stemming=True)

display(result)

,preprocessed_full_description,full_description
27195,20 livr condens volont vivr ami flick miracl l...,20 Livres Condenses : Volonte De Vivre Mon Ami...
44069,building act regul applied,The Building Acts And Regulations Applied
28796,up-grad cello grad 3-5,Up-Grade! Cello Grades 3-5
46072,a/r andat + ritorno,A/R Andata + Ritorno
44781,j. m. coetze lif writing face-to-fac tim,J. M. Coetzee And The Life Of Writing: Face-To...
49512,women ' fiction second world war,Women's Fiction Of The Second World War
34848,ye hom go,Have Ye No Homes To Go To?
47043,roi san divert,Un Roi Sans Divertissement
42277,splendeur l'orient 1.le temp invas collect gra...,Splendeur De L'orient - 1.Le Temps Des Invasio...
28190,vulgar pupillar substitution droit romain prés...,De Vulgari Et Pupillari Substitutione Droit Ro...


On note avec l'extrait ci-dessus que la stemmatisation est trop violente sur le contenu de la variable d'une part et qu'il n'est pas adapté sur un contenu multilingue tel que nous avons ici (français, anglais mais aussi parfois allemand). Afin de limiter la perte en information, nous décidons de ne pas appliquer de lemmatisation ni de stemmatisation.

Il aurait aussi été possible de traduire les textes pour uniformiser la langue employée mais cela représente :
- un traitement coûteux
- un risque de perte d'information si la traduction n'est pas de bonne qualité

Comme le jeu de données est conséquent, on peut s'attendre à ce que les algorithme d'analyse textuel arrivent à extraire l'information utile malgré la présence de multiple langues.

On décide donc de limiter le prétraitement textuel aux éléments suivants :
- nettoyage des stopwords anglais et français
- nettoyage de la ponctuation
- tokenisation des chaînes de caractères

In [7]:
X_train['preprocessed_full_description'] = ""
X_test['preprocessed_full_description'] = ""

for i in X_train.index:
    X_train.loc[i, 'preprocessed_full_description'] = preprocessing(X_train.loc[i, 'full_description'])

for i in X_test.index:
    X_test.loc[i, 'preprocessed_full_description'] = preprocessing(X_test.loc[i, 'full_description'])

In [8]:
display(X_train[["full_description", "preprocessed_full_description"]].head(10))
display(X_test[["full_description", "preprocessed_full_description"]].head(10))

,full_description,preprocessed_full_description
27195,20 Livres Condenses : Volonte De Vivre Mon Ami...,20 livres condenses volonte vivre amie flicka ...
44069,The Building Acts And Regulations Applied,building acts regulations applied
28796,Up-Grade! Cello Grades 3-5,up-grade cello grades 3-5
46072,A/R Andata + Ritorno,a/r andata + ritorno
44781,J. M. Coetzee And The Life Of Writing: Face-To...,j. m. coetzee life writing face-to-face time
49512,Women's Fiction Of The Second World War,women 's fiction second world war
34848,Have Ye No Homes To Go To?,ye homes go
47043,Un Roi Sans Divertissement,roi sans divertissement
42277,Splendeur De L'orient - 1.Le Temps Des Invasio...,splendeur l'orient 1.le temps invasions collec...
28190,De Vulgari Et Pupillari Substitutione Droit Ro...,vulgari pupillari substitutione droit romain p...


,full_description,preprocessed_full_description
49891,Intex Flotteur gonflable pour piscine Flamingo...,intex flotteur gonflable piscine flamingo part...
250,Motif Géométrique Irrégulière Coussin Coussin ...,motif géométrique irrégulière coussin coussin ...
64385,HTH Alkanal poudre - 5 kg Alkanal augmente l'a...,hth alkanal poudre 5 kg alkanal augmente l'alc...
6657,Carnet De Notes Bloc-Notes Cahier L'amour De L...,carnet notes bloc-notes cahier l'amour paix se...
41121,Piscine Gonflable Ovale 262x160x46 Cm Intex - ...,piscine gonflable ovale 262x160x46 cm intex pi...
52922,1 Pièce/Boîte Portable Classique 1 Gb 2.7 Pouc...,1 pièce/boîte portable classique 1 gb 2.7 pouc...
57430,Simple Linen Creative Belle Oreiller Taie D'or...,simple linen creative belle oreiller taie d'or...
15288,Halloween Accueil Voiture Lit Lettre Décorativ...,halloween accueil voiture lit lettre décorativ...
77056,Console &amp; Jeux Wii,console & amp jeux wii
44137,Suspension 3 Lumieres Corinna Ce suspension tr...,suspension 3 lumieres corinna suspension trois...


### Application de TF-IDF à la variable `full_description`

En l'état, les chaînes de caractères dans full_description sont difficilement exploitables par des algorithmes de machine learning.

Il est nécessaire de les transformer en valeurs numériques interprétables. Nous avons décidé d'appliquer le TF-IDF pour une extraction de donnée rapide et efficace suite à notre analyse en wordcloud qui fait apparaître la répétition de mots clés pour chaque catégorie de produits.

In [9]:

# Initialisation du vecteur TF-IDF
tfidf = TfidfVectorizer(
    max_features=10000,
    ngram_range=(1,2)
)

# Apprentissage sur X_train et transformation
X_train_tfidf = tfidf.fit_transform(X_train['preprocessed_full_description'])
X_test_tfidf = tfidf.transform(X_test['preprocessed_full_description'])

# Suppression des colones temporaires
X_train = X_train.drop(columns=['preprocessed_full_description', 'full_description'])
X_test = X_test.drop(columns=['preprocessed_full_description', 'full_description'])

# Concaténation
X_train_final = hstack([X_train, X_train_tfidf])
X_test_final = hstack([X_test, X_test_tfidf])

### Sauvegarde des données textuelles suite à leur préprocessing

Afin de faciliter le travail sur plusieurs modèles et gagner du temps, on décide de sauvegarder les données prétraitées.

In [10]:
# Sauvegarde des données prétraitées dans des fichiers CSV

X_train.to_csv("../data/preprocessed/X_train_preprocessed.csv")
y_train.to_csv("../data/preprocessed/y_train_preprocessed.csv")
X_test.to_csv("../data/preprocessed/X_test_preprocessed.csv")
y_test.to_csv("../data/preprocessed/y_test_preprocessed.csv")


## Preprocessing des données graphiques